In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn tools
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# Loading in the dataset
spotify = pd.read_csv("spotify.csv")

# Dropping unnecessary columns
spotify = spotify.drop(columns=['Unnamed: 0', 'song_title', 'artist', 'target'])

# Defining the predictors and target
X = spotify.drop(columns=['valence'])
y = spotify['valence']

# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
# Define categorical and numeric columns
cat_cols = ['mode', 'key']
num_cols = ['energy', 'danceability', 'loudness', 'speechiness', 'acousticness']

# Creating a column transformer
transformer = make_column_transformer(
    (OneHotEncoder(drop='first'), cat_cols),
    (StandardScaler(), num_cols)
)

# Pipeline with transformer and regression model
pipe = Pipeline([
    ('preprocess', transformer),
    ('model', LinearRegression())
])

# Fitting the model
pipe.fit(X_train, y_train)

# Predicting and evaluating
y_pred = pipe.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)


#Outputting
print(f"Pipeline Linear Regression MSE: {mse:.6f}")
print(f"Pipeline Linear Regression RMSE: {rmse:.6f}")


Pipeline Linear Regression MSE: 0.046473
Pipeline Linear Regression RMSE: 0.215576


# STREAMLIT APP

In [4]:
!pip install -q streamlit
!npm install -g localtunnel
import joblib

# Saving the trained pipeline
joblib.dump(pipe, 'linear_valence_model.pkl')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

['linear_valence_model.pkl']

In [5]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib

# Loading the trained model
model = joblib.load('linear_valence_model.pkl')

# Streamlit app
st.title("🎵 Spotify Valence Predictor")
st.markdown("Predict the emotional valence of a song based on its audio features.")

# Inputting fields for all required features
danceability = st.slider("Danceability", 0.0, 1.0, 0.5)
energy = st.slider("Energy", 0.0, 1.0, 0.5)
acousticness = st.slider("Acousticness", 0.0, 1.0, 0.5)
speechiness = st.slider("Speechiness", 0.0, 1.0, 0.5)  # Added
loudness = st.slider("Loudness (dB)", -60.0, 0.0, -20.0)  # Added
tempo = st.slider("Tempo (BPM)", 40.0, 220.0, 120.0)
key = st.selectbox("Key (0=C, ..., 11=B)", list(range(12)))
mode = st.selectbox("Mode (0=Minor, 1=Major)", [0, 1])

# Creating input DataFrame with all expected features
input_data = pd.DataFrame({
    'danceability': [danceability],
    'energy': [energy],
    'acousticness': [acousticness],
    'speechiness': [speechiness],       # Match training features
    'loudness': [loudness],
    'tempo': [tempo],
    'key': [key],
    'mode': [mode]
})

# Predicting valence
if st.button("Predict Valence"):
    prediction = model.predict(input_data)[0]
    st.success(f"Predicted Valence: {prediction:.3f}")


Writing app.py


In [6]:
!streamlit run app.py &>/content/logs.txt &


In [8]:
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦your url is: https://heavy-rooms-run.loca.lt
^C


In [7]:
import urllib.request
print("Your public IP:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip())


Your public IP: 35.194.93.181
